In [14]:
from pathlib import Path
import sys

import pandas as pd
import numpy as np

sys.path.append('../')
import payulator as pl


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
loan = pl.Loan(interest_rate=0.12, fee=250)
display(loan)
loan.summarize()['payment_schedule']

Loan(code='loan-2019-04-15T11:13:54', kind='amortized', principal=1, interest_rate=0.12, payment_freq='monthly', compounding_freq='monthly', num_payments=1, fee=250, first_payment_date=None)

,payment_sequence,beginning_balance,principal_payment,interest_payment,fee_payment,ending_balance,notes
0,1,1.0,1.0,0.01,250,0.0,NaN


In [31]:
params = {
    'code': 'test-20180324',
    'principal': 10000,
    'interest_rate': 0.12,
    'kind': 'amortized',
    'num_payments': 12,
    'payment_freq': 'monthly',
    'first_payment_date': '2018-03-06',
    'fee': 250,
}

params = pl.check_loan_params(params)
loan = pl.Loan(**params)
loan.summarize()

{'periodic_payment': 888.49,
 'payment_schedule':     payment_sequence payment_date  beginning_balance  principal_payment  \
 0                  1   2018-03-06           10000.00             788.49   
 1                  2   2018-04-06            9211.51             796.37   
 2                  3   2018-05-06            8415.14             804.34   
 3                  4   2018-06-06            7610.80             812.38   
 4                  5   2018-07-06            6798.42             820.50   
 5                  6   2018-08-06            5977.92             828.71   
 6                  7   2018-09-06            5149.21             837.00   
 7                  8   2018-10-06            4312.21             845.37   
 8                  9   2018-11-06            3466.85             853.82   
 9                 10   2018-12-06            2613.03             862.36   
 10                11   2019-01-06            1750.67             870.98   
 11                12   2019-02-06     

In [28]:
loan.summarize()["payment_schedule"]

,payment_sequence,payment_date,beginning_balance,principal_payment,ending_balance,interest_payment,fee_payment,notes
0,1,2018-03-06,10000,0,10000,100.0,250,NaN
1,2,2018-04-06,10000,0,10000,100.0,0,NaN
2,3,2018-05-06,10000,0,10000,100.0,0,NaN
3,4,2018-06-06,10000,0,10000,100.0,0,NaN
4,5,2018-07-06,10000,0,10000,100.0,0,NaN
5,6,2018-08-06,10000,0,10000,100.0,0,NaN
6,7,2018-09-06,10000,0,10000,100.0,0,NaN
7,8,2018-10-06,10000,0,10000,100.0,0,NaN
8,9,2018-11-06,10000,0,10000,100.0,0,NaN
9,10,2018-12-06,10000,0,10000,100.0,0,NaN


In [66]:
def sum_payments(loans, start_date: str=None, end_date: str=None, freq="M"):
    """
    Aggregate the payment schedules of the given loans using the function
    :func:`payulator.aggregate_payment_schedules` at the given summary frequency.
    Slice the result to the given start and end dates (inclusive), 
    append a few summary columns, and return the resulting DataFrame
    with the columns
    
    - ``"payment_date"``    
    - ``"fee_payment"``
    - ``"principal_payment"``   
    - ``"interest_payment"``    
    - ``"total_payment"``
    - ``"fee_payment_cumsum": cumulative sum of fee_payment
    - ``"principal_payment_cumsum"``: cumulative sum of principal_payment  
    - ``"interest_payment_cumsum"``: cumulative sum of interest_payment    
    - ``"total_payment_cumsum"``: cumulative sume of total_payment
    
    """
    # Collect payment schedules from all loans on record
    s = [loan.summarize()["payment_schedule"] for loan in loans]
    if not s:
        return pd.DataFrame()
    
    # Aggregate
    return (
        pl.aggregate_payment_schedules(s, freq=freq)
        .set_index("payment_date")
        .loc[start_date: end_date]
        .assign(principal_payment_cumsum=lambda x: x.principal_payment.cumsum())
        .assign(interest_payment_cumsum=lambda x: x.interest_payment.cumsum())
        .assign(fee_payment_cumsum=lambda x: x.fee_payment.cumsum())
        .assign(total_payment_cumsum=lambda x: x.total_payment.cumsum())
        .reset_index()
    )


In [83]:
loans = [loan, loan]
s = [loan.summarize()["payment_schedule"] for loan in loans]
f = pl.aggregate_payment_schedules(s, start_date="2018-04-06", freq=None)
f.T

,0,1,2,3,4,5,6,7,8,9,10
payment_date,2018-04-06 00:00:00,2018-05-06 00:00:00,2018-06-06 00:00:00,2018-07-06 00:00:00,2018-08-06 00:00:00,2018-09-06 00:00:00,2018-10-06 00:00:00,2018-11-06 00:00:00,2018-12-06 00:00:00,2019-01-06 00:00:00,2019-02-06 00:00:00
principal_payment,1592.74,1608.68,1624.76,1641,1657.42,1674,1690.74,1707.64,1724.72,1741.96,1759.38
interest_payment,184.24,168.3,152.22,135.96,119.56,102.98,86.24,69.34,52.26,35.02,17.6
fee_payment,0,0,0,0,0,0,0,0,0,0,0
total_payment,1776.98,1776.98,1776.98,1776.96,1776.98,1776.98,1776.98,1776.98,1776.98,1776.98,1776.98
principal_payment_cumsum,1592.74,3201.42,4826.18,6467.18,8124.6,9798.6,11489.3,13197,14921.7,16663.7,18423
interest_payment_cumsum,184.24,352.54,504.76,640.72,760.28,863.26,949.5,1018.84,1071.1,1106.12,1123.72
fee_payment_cumsum,0,0,0,0,0,0,0,0,0,0,0
total_payment_cumsum,1776.98,3553.96,5330.94,7107.9,8884.88,10661.9,12438.8,14215.8,15992.8,17769.8,19546.8


In [82]:
f.dtypes

payment_date                datetime64[ns]
principal_payment                  float64
interest_payment                   float64
fee_payment                          int64
total_payment                      float64
principal_payment_cumsum           float64
interest_payment_cumsum            float64
fee_payment_cumsum                   int64
total_payment_cumsum               float64
dtype: object